In [11]:
import torch
import torch.nn as nn
import torch.nn.functional as F

forward 함수만 정의하고 나면, (변화도를 계산하는) backward 함수는 autograd 를 사용하여 자동으로 정의됩니다. forward 함수에서는 어떠한 Tensor 연산을 사용해도 됩니다.

In [12]:
class Net(nn.Module):

    def __init__(self):
        super(Net, self).__init__()
        # 입력 이미지 채널 1개, 출력 채널 6개, 3x3의 정사각 컨볼루션 행렬
        # 컨볼루션 커널 정의
        self.conv1 = nn.Conv2d(1, 6, 3)
        self.conv2 = nn.Conv2d(6, 16, 3)
        # 아핀(affine) 연산: y = Wx + b
        self.fc1 = nn.Linear(16 * 6 * 6, 120)  # 6*6은 이미지 차원에 해당
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        # (2, 2) 크기 윈도우에 대해 맥스 풀링(max pooling)
        x = F.max_pool2d(F.relu(self.conv1(x)), (2, 2))
        # 크기가 제곱수라면 하나의 숫자만을 특정
        x = F.max_pool2d(F.relu(self.conv2(x)), 2)
        x = x.view(-1, self.num_flat_features(x))
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

    def num_flat_features(self, x):
        size = x.size()[1:]  # 배치 차원을 제외한 모든 차원
        num_features = 1
        for s in size:
            num_features *= s
        return num_features


net = Net()
print(net)

Net(
  (conv1): Conv2d(1, 6, kernel_size=(3, 3), stride=(1, 1))
  (conv2): Conv2d(6, 16, kernel_size=(3, 3), stride=(1, 1))
  (fc1): Linear(in_features=576, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)


In [13]:
params = list(net.parameters())
print(len(params))



print(params[0].size()) # conv1의 .weight

10
torch.Size([6, 1, 3, 3])


In [16]:
input = torch.randn(1, 1, 32, 32)
print(input)
out = net(input)
out

tensor([[[[-1.4400, -0.7015, -0.2239,  ..., -0.8976, -2.1916,  0.1515],
          [ 0.1641, -0.8967,  0.8724,  ..., -0.4541,  1.8663, -1.2534],
          [ 0.8197,  1.4773,  0.4521,  ...,  0.8905, -1.4283, -0.0716],
          ...,
          [-0.6927,  0.5910, -0.4864,  ...,  0.4050,  1.3737, -1.7011],
          [-0.6432, -0.5782, -0.8498,  ...,  0.3621,  0.3200,  1.1541],
          [ 0.5465, -0.4397,  0.2430,  ..., -0.8013,  1.1338,  0.7170]]]])


tensor([[ 0.0140, -0.1008,  0.0348, -0.0571, -0.0380, -0.0078, -0.0575, -0.0691,
          0.0804, -0.0798]], grad_fn=<AddmmBackward>)

In [15]:
net.zero_grad()
out.backward(torch.randn(1,10))

In [17]:
output = net(input)
target = torch.randn(10) # 예시를 위한 임의의 정답
target = target.view(1, -1)
criterion = nn.MSELoss()

loss = criterion(output, target)
loss

tensor(0.5572, grad_fn=<MseLossBackward>)

In [21]:
print(loss.grad_fn)
print(loss.grad_fn.next_functions[0][0])
print(loss.grad_fn.next_functions[0][0].next_functions[0][0])

In [22]:
net.zero_grad()
print(net.conv1.bias.grad)

loss.backward() # 역전파 -> 값 초기화됨
print(net.conv1.bias.grad)

tensor([0., 0., 0., 0., 0., 0.])
tensor([-0.0058,  0.0124, -0.0081, -0.0075, -0.0019,  0.0106])


- sgd
 - 새로운 가중치(weight) = 가중치(weight) - 학습률(learning rate) * 변화도(gradient)

In [23]:
learning_rate = 0.01
for f in net.parameters():
    f.data.sub_(f.grad.data * learning_rate)

In [24]:
import torch.optim as optim
optimizer = optim.SGD(net.parameters(), lr=0.01)

# 학습 과정(training loop)
optimizer.zero_grad()
output = net(input)
loss = criterion(output, target)
loss.backward()
optimizer.step()